In [ ]:
import os

# Replace this with the path to your main folder
root_folder = "D:\Drive D\models"

# Get all files ending with "lr0001.h5" in all subfolders
model_paths = [
    os.path.join(dirpath, filename)
    for dirpath, dirnames, filenames in os.walk(root_folder)
    for filename in filenames
    if filename.endswith("lr0001.h5")
]

# Now you can iterate through these files
print(f"There {len(model_paths)} models in {root_folder}")
for file_path in model_paths:
    print(file_path)
    # Add your processing code here

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
import pandas as pd
import time
import numpy as np

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

data_dir = r"C:\Users\kanan\Desktop\data_x\data_x\test"
test = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=(224, 224))